In [1]:
import pandas as pd 
# Take the following dataset: (Combindation of EIA-860 and EIA-923 Data)
df = pd.read_excel('Solar Workbook.xlsx', sheet_name = 'Combined Data for Project') 

In [2]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np 

# Feature column titles for generation 
genList = ["Netgen January", "Netgen February", "Netgen March", "Netgen April", "Netgen May", 
            "Netgen June", "Netgen July", "Netgen August", "Netgen September", "Netgen October", 
           "Netgen November", "Netgen December"]

# Clean the data 
df.columns = [x.replace("\n", " ") for x in df.columns.to_list()]
df[genList] = df[genList].replace(['.'], '0')
df[genList] = df[genList].astype(int) 

# Drop any row with a zero generation for any month 
df = df.loc[(df[genList] != 0).all(axis = 1)] 

# Drop the state of Hawaii 
df.drop(df.index[df['State'] == "HI"], inplace = True)

# Obtain a dataframe of the generations 
gen = df[genList] 



# Nameplate Capacity is the best estimate for size of the plant. It represents maximum amount of energy a plant could generate 
# Adjust all the plants generation for size and the number of days per month 
nameplateCapacity = df["Nameplate Capacity (MW)"] 
daysInMonth = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

# Apply formula for efficiency: 
for i in range(12): 
    gen.iloc[:,i] = (gen.iloc[:,i]).div(nameplateCapacity) 
    gen.iloc[:,i] = gen.iloc[:,i].div(daysInMonth[i])

# Normalize the efficiencies to values between -1 and 1 
gen = (gen - gen.mean()) / gen.std()

# Print the shape: 
print(gen.shape) 


print("Completed")

(3347, 12)
Completed


/Users/christopherschenk/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [3]:
from scipy import stats
# Lets just keep: Location and Efficiency  
dataFrames = [gen, df["Latitude"], df["Longitude"]]   
df = pd.concat(dataFrames, axis = 1) 

# Find this dataframe to excel: 
df.to_excel('AidToSearchPVLocation&Efficiency.xlsx')

# Now, go to the site https://wwwx.prism.oregonstate.edu/historical/ , select all weather options and download the data in groups of 500 directly from the PVLocationEfficiency.xlsx excel sheet. 
# Or you can set up an FTP here: https://www.prism.oregonstate.edu/downloads/ 

In [4]:
import pandas as pd
import numpy as np 

# To manually import data use an excel sheet labeled LocationWeatherData and have each sheet be labeled as a set# as shown below. 
# Make sure to remove the first row, or set it to 1. 

a1 = pd.read_excel("LocationWeatherData.xlsx", sheet_name = "set1").to_numpy()
a2 = pd.read_excel("LocationWeatherData.xlsx", sheet_name = "set2").to_numpy()
a3 = pd.read_excel("LocationWeatherData.xlsx", sheet_name = "set3").to_numpy()
a4 = pd.read_excel("LocationWeatherData.xlsx", sheet_name = "set4").to_numpy()
a5 = pd.read_excel("LocationWeatherData.xlsx", sheet_name = "set5").to_numpy()
a6 = pd.read_excel("LocationWeatherData.xlsx", sheet_name = "set6").to_numpy()
a7 = pd.read_excel("LocationWeatherData.xlsx", sheet_name = "set7").to_numpy()

arrays = [a2, a3, a4, a5, a6, a7]
data = a1

for array in arrays: 
    data = np.concatenate([data, array]) 

data = data[:, 5:] 

data = pd.DataFrame(data) 
data = data.dropna(axis = 0)

data = data.to_numpy() 


In [5]:
data2 = np.reshape(data, (3347, 84)) 
print(data2[1,:]) 
# Now you have the monthly weather data for every solar power plant in the US that runs all year around 

[2.54 20.9 27.3 33.7 22.3 0.35 1.82 0.63 20.5 28.4 36.2 19.7 0.66 3.33
 3.07 32.4 41.2 50.0 32.1 0.6 5.8 2.33 37.9 48.6 59.3 35.6 1.02 10.13 4.96
 50.3 59.7 69.0 47.9 1.18 12.9 4.81 62.0 73.3 84.5 58.6 2.1 23.35 3.77
 66.7 76.8 86.9 67.3 0.95 20.64 1.07 61.9 73.2 84.5 61.8 1.37 21.67 8.5
 53.8 64.1 74.5 53.8 1.14 15.2 2.41 39.2 49.7 60.2 37.0 1.26 11.71 3.03
 35.7 45.9 56.2 34.2 1.5 8.63 2.89 21.4 29.7 38.0 22.4 0.45 3.29]


In [6]:
# Add the correct titles to this data: 
titles = np.array(['PPT Month 1', 'Tmin Month 1', 'Tmean Month 1', 'Tmax Month 1', 'TDmean Month 1', 'VPDmin Month 1', 'VPDmax Month 1', 
          'PPT Month 2', 'Tmin Month 2', 'Tmean Month 2', 'Tmax Month 2', 'TDmean Month 2', 'VPDmin Month 2', 'VPDmax Month 2', 
          'PPT Month 3', 'Tmin Month 3', 'Tmean Month 3', 'Tmax Month 3', 'TDmean Month 3', 'VPDmin Month 3', 'VPDmax Month 3', 
          'PPT Month 4', 'Tmin Month 4', 'Tmean Month 4', 'Tmax Month 4', 'TDmean Month 4', 'VPDmin Month 4', 'VPDmax Month 4', 
          'PPT Month 5', 'Tmin Month 5', 'Tmean Month 5', 'Tmax Month 5', 'TDmean Month 5', 'VPDmin Month 5', 'VPDmax Month 5', 
          'PPT Month 6', 'Tmin Month 6', 'Tmean Month 6', 'Tmax Month 6', 'TDmean Month 6', 'VPDmin Month 6', 'VPDmax Month 6', 
          'PPT Month 7', 'Tmin Month 7', 'Tmean Month 7', 'Tmax Month 7', 'TDmean Month 7', 'VPDmin Month 7', 'VPDmax Month 7', 
          'PPT Month 8', 'Tmin Month 8', 'Tmean Month 8', 'Tmax Month 8', 'TDmean Month 8', 'VPDmin Month 8', 'VPDmax Month 8', 
          'PPT Month 9', 'Tmin Month 9', 'Tmean Month 9', 'Tmax Month 9', 'TDmean Month 9', 'VPDmin Month 9', 'VPDmax Month 9', 
          'PPT Month 10', 'Tmin Month 10', 'Tmean Month 10', 'Tmax Month 10', 'TDmean Month 10', 'VPDmin Month 10', 'VPDmax Month 10', 
          'PPT Month 11', 'Tmin Month 11', 'Tmean Month 11', 'Tmax Month 11', 'TDmean Month 11', 'VPDmin Month 11', 'VPDmax Month 11', 
          'PPT Month 12', 'Tmin Month 12', 'Tmean Month 12', 'Tmax Month 12', 'TDmean Month 12', 'VPDmin Month 12', 'VPDmax Month 12', 
         ])
weatherData = pd.DataFrame(columns = titles, data = data2) 

In [7]:
from scipy import stats
# Lets just keep: Efficiency and Location 
dataFrames = [df["Latitude"], df["Longitude"], gen] 

# Group these two dataFrames together  
df = pd.concat(dataFrames, axis = 1) 

# Switch to numpy 
df = df.to_numpy() 

# Match the weather data: 
# Units: ppt (inches)	tmin (degrees F)	tmean (degrees F)	tmax (degrees F)	tdmean (degrees F)	vpdmin (hPa)	vpdmax (hPa)
weatherData = weatherData.iloc[: , 1:] # The first column is just indices, ignore t
weatherColumnTitles = weatherData.columns.values.tolist() 
weatherData = weatherData.to_numpy() 

# Clean up the titles 
newTitles = ["Latitude", "Longitude", "Efficiency January", "Efficiency February", "Efficiency March", "Efficiency April", "Efficiency May", 
            "Efficiency June", "Efficiency July", "Efficiency August", "Efficiency September", "Efficiency October", 
           "Efficiency November", "Efficiency December"]
newTitles.extend(weatherColumnTitles) 

# Add the weather data to the dataFrame 
df = np.hstack((df, weatherData)) 
df = pd.DataFrame(df, columns = newTitles) 

df.to_excel("solarEfficiencyAndWeather2020.xlsx") 

print(df) 

      Latitude Longitude Efficiency January Efficiency February  \
0      27.7944  -82.4036           1.062897            0.747417   
1     41.45149  -90.1485          -0.947942           -0.534998   
2      41.9093  -89.0498          -0.138702           -0.738785   
3     43.27972  -92.8106          -1.453718           -0.693499   
4     37.78831  -84.7126          -0.781426           -1.276642   
...        ...       ...                ...                 ...   
3342  34.01964  -85.2809           0.808477            0.208103   
3343  42.22737  -71.2489          -0.179164           -0.095725   
3344  41.67901  -71.1111          -0.142748           -0.041382   
3345  41.70463  -71.0897          -0.138702           -0.038363   
3346  41.40755  -71.7483          -0.536773           -0.379755   

     Efficiency March Efficiency April Efficiency May Efficiency June  \
0            1.270012         0.171215       0.592278       -0.033252   
1           -1.203772        -0.706027      -1.19